In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#Loading Data and Scaling (divide by 255)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train  = x_train / 255
x_test = x_test / 255
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [3]:
#Flattening X(Features)
x = x_train.reshape(x_train.shape[0], x_train.shape[1] ** 2)
x = tf.constant(x, dtype=tf.float32)
x.shape

TensorShape([60000, 784])

In [4]:
#Y(labels)
#y = np.zeros((y_train.size, y_train.max()+1))
#y[np.arange(y_train.size),y_train] = 1
y = tf.constant(y_train, dtype=tf.float32)
y.shape

TensorShape([60000])

In [5]:
def adamOptimizer(dxs, first_moments, second_moments, t):
    dxc = []
    fm = []
    sm = []
    for dx, first_moment, second_moment in zip(dxs, first_moments, second_moments):
        first_moment = 0.9 * first_moment + (1-0.9) * dx
        second_moment = 0.999 * second_moment + (1-0.999) * dx * dx
        first_unbias = first_moment / (1 - 0.9 ** t)
        second_unbias = second_moment/ (1 - 0.999 ** t)
        dxc.append(first_unbias / (tf.sqrt(second_unbias) + 1e-7))
        fm.append(first_moment)
        sm.append(second_moment)
    return dxc, fm, sm

def batchNorm(x):
    mean = tf.reduce_mean(x, axis=0)
    var = tf.math.reduce_variance(x, axis=0)
    return (x - mean) / tf.math.sqrt(var+1e-7)

In [6]:
w1 = tf.Variable(tf.random.normal((784, 784), dtype=tf.float32))
b1 = tf.Variable(tf.random.normal((1, 784), dtype=tf.float32))
w2 = tf.Variable(tf.random.normal((784, 784), dtype=tf.float32))
b2 = tf.Variable(tf.random.normal((1, 784), dtype=tf.float32))
w3 = tf.Variable(tf.random.normal((784, 10), dtype=tf.float32))
b3 = tf.Variable(tf.random.normal((1, 10), dtype=tf.float32))

In [7]:
ht_p = tf.zeros((60000, 784), dtype = tf.float32)
fm = [0,0,0,0,0,0]
sm = [0,0,0,0,0,0]
lr = 0.1
itr = 10
for j in range(itr+1):
  for i in range(784):
    with tf.GradientTape() as g:
      g.watch([w1, b1, w2, b2, w3, b3])
      mask = np.zeros(x.shape)
      mask[:,205] = i
      x_f_temp = tf.where(mask, x, tf.zeros(x.shape))
      ht = tf.matmul(x_f_temp, w1) + b1
      ot = tf.matmul(ht_p, w2) + b2
      ht_p = ht
      ht_ot = ht + ot
      a_t = tf.matmul(ht_ot, w3) + b3
      y_pred = tf.nn.softmax(a_t)
      ce = tf.keras.losses.sparse_categorical_crossentropy(y, y_pred)
      loss = tf.reduce_mean(ce)
      
      dw1, db1, dw2, db2, dw3, db3 = g.gradient(loss, [w1, b1, w2, b2, w3, b3])
      
      #Adam Optimizer
      (dw1, db1, dw2, db2, dw3, db3) , fm, sm = adamOptimizer([dw1, db1, dw2, db2, dw3, db3], fm, sm, i+1)

      #Gradient Update
      w1.assign_sub(lr*dw1)
      b1.assign_sub(lr*db1)
      w2.assign_sub(lr*dw2)
      b2.assign_sub(lr*db2)
      w3.assign_sub(lr*dw3)
      b3.assign_sub(lr*db3)

  print(j, loss.numpy())

0 14.435098
1 14.435098
2 14.435098
3 14.435098
4 14.435098


KeyboardInterrupt: ignored